![HSV-AI Logo](https://github.com/HSV-AI/hugo-website/blob/master/static/images/logo_v9.png?raw=true)

# Implicit Recommendation from ECommerce Data

Some of the material for this work is based on [A Gentle Introduction to Recommender Systems with Implicit Feedback](https://jessesw.com/Rec-System/) by Jesse Steinweg Woods. This tutorial includes an implementation of the Alternating Least Squares algorithm and some other useful functions (like the area under the curve calculation). Other parts of the tutorial are based on a previous version of the Implicit library and had to be reworked.


## Basics of EDA

Here are a few things that we are looking for in the invoice / transaction data:

1. Were there any negative totals? If so why?
2. What percentage of the purchases actually contained multiple items?
3. What is the spread of purchases by customer ID? Do we have a few customers whose behavior may drive recommendations in a way that doesn't fit the average customer?
4. Where there any purchases that were VERY large? If so why? Do we want to include these values to train model behavior?
5. Is there any missing data that we need to scrub?


In [ ]:
%reload_kedro

In [ ]:
import pandas as pd
import numpy as np
import random
import scipy.sparse
from matplotlib import pyplot as plt
from pandas.api.types import CategoricalDtype


In [ ]:
transactions = catalog.load("brazillian_kaggle_order_data")
products = catalog.load("brazillian_kaggle_product_data")
customers = catalog.load("brazillian_kaggle_customer_data")

In [ ]:
transactions['order_id'] = transactions.order_id.astype(str)
transactions['product_id'] = transactions.product_id.astype(str)
transactions.head()

In [ ]:
products['product_id'] = products.product_id.astype(str)
products.head()

In [ ]:
customers.head()

In [ ]:
customers = customers[["customer_id", "order_id", "order_purchase_timestamp"]]
customers['order_id'] = customers.order_id.astype(str)
customers['customer_id'] = customers.customer_id.astype(str)

customers.head()

# Data Overview

In [ ]:
order_count = len(pd.unique(transactions['order_id']))
product_count = len(pd.unique(transactions['product_id']))
customer_count = len(pd.unique(customers['customer_id']))

output = """
    The dataset includes:
    * {order_count} orders
    * {customer_count} customers
    * {product_count} products
""".format(order_count=order_count, product_count=product_count, customer_count=customer_count)
print(output)

# Transactions over Time

Now we need to look at the number of items purchased each day to see if there is anything interesting that pops out.

In [ ]:
from datetime import datetime

datetime_object = datetime.strptime('2017-10-02 10:56:33', '%Y-%m-%d %H:%M:%S')

In [ ]:
from datetime import datetime

datetime_object = datetime.strptime('Mon Feb 12 04:26:00 IST 2018', '%a %b %d %H:%M:%S IST %Y')

def func(date):
    temp = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
    return temp.strftime('%Y%m%d')

transactions['datetime'] = transactions.apply(lambda x: func(x.order_purchase_timestamp), axis=1)


In [ ]:
transactions.groupby(['datetime'])['datetime'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['datetime'], ascending=True) \
                             .plot(figsize=(15,10))

# Data Sparcity

Let's take a look at the sparcity of the data. This will tell us how many products were purchased across multiple orders. This is directly related to how well a recommendation system can be trained.

In [ ]:
transaction_list = list(np.sort(filtered_df.order_id.unique())) # Get our unique customers
item_list = list(filtered_df.product_id.unique()) # Get our unique products that were purchased
quantity_list = list(filtered_df.quantity) # All of our purchases

cols = filtered_df.order_id.astype(CategoricalDtype(categories=transaction_list, ordered=True)).cat.codes 
# Get the associated row indices
rows = filtered_df.product_id.astype(CategoricalDtype(categories=item_list, ordered=True)).cat.codes 
# Get the associated column indices
purchases_sparse = scipy.sparse.csr_matrix((quantity_list, (rows, cols)), shape=(len(item_list), len(transaction_list)))

In [ ]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

In [ ]:
plt.figure(figsize=(15, 15))
plt.spy(purchases_sparse, markersize=1, aspect='auto')

# Storing Interim Data

Now that we have the data cleaned up a bit and formatted correctly, we can save it to an interim file to be picked up by the model training algorithm.

In [ ]:
catalog.save("brazillian_transactions", filtered_df[["order_id", "product_id", "price", "quantity"]])

products["description"] = products["product_category_name"] + str(products["product_description_lenght"])

catalog.save("brazillian_products", products[["product_id", "description"]])